<a href="https://colab.research.google.com/github/Struth-Rourke/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/Assignment_(Working)_RNN_and_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
import requests
import pandas as pd

In [ ]:
# URL
url = "https://www.gutenberg.org/files/100/100-0.txt"

# Requests the infor via '.get'
r = requests.get(url)
# Encoding 'r'
r.encoding = r.apparent_encoding
# Retrieving the 'text' column of the data
data = r.text
# Splitting data on specified characters
data = data.split('\r\n')
# Stripping the ends of whitespace
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Specific Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

# Adding locations of the play titles
locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            
# Creating a DF
df_toc = pd.DataFrame.from_dict(locations, orient='index')
# Start and End w/ Character Locations 
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
# Finding the 'len' of 'data'
df_toc.loc[42, 'end'] = len(data)
# Setting 'end' len typye as an 'int'
df_toc['end'] = df_toc['end'].astype('int')

# Joining, and creating the columns together in the DF
df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [ ]:
# Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [ ]:
import re

# Cleaning the text of 
def text_cleaner(text):
  text = re.sub("^ï»¿", "", text) # no extra characters at the start
  text = re.sub("[\r\n]", " ", text) # no escape characters
  text = re.sub("â.{2}", "", text) # no byte escapes
  text = re.sub(" +", " ", text) # standardize spacings
  return text

# TEST: Calling function on text
df_toc["text_cleaned"] = df_toc["text"].apply(text_cleaner)

# CHECK:
print(df_toc.shape)
df_toc.head()


(43, 5)


,title,start,end,text,text_cleaned
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...,ALL’S WELL THAT ENDS WELL Contents ACT I Scene...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...,THE TRAGEDY OF ANTONY AND CLEOPATRA DRAMATIS P...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...,"AS YOU LIKE IT DRAMATIS PERSONAE. DUKE, living..."
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...,THE COMEDY OF ERRORS Contents ACT I Scene I. A...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...,THE TRAGEDY OF CORIOLANUS Dramatis Personae CA...


In [ ]:
# Grabbing Entire DataSet
text = requests.get('https://www.gutenberg.org/files/100/100-0.txt')

# Cleaning Function
def shakespeare_cleaner(text):
  text = text.text
  text = re.sub('^ï»¿', '', text) # no extra guff at the start
  text = re.sub('[\r\n]', ' ', text) # no escape characters
  text = re.sub('â.{2}', '', text) # no byte escapes
  text = re.sub(' +', ' ', text) # standardize spacings
  return text

# Calling Function
cleaned_text = shakespeare_cleaner(text)

In [ ]:
import numpy as np

# Unique Characters
chars = list(set(cleaned_text))

# Lookup tables
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

# Generate sequence data
max_len = 40
step = 5

# Encoding
encoded = [char_int[c] for c in cleaned_text]

sequences = [] # Each element is max_len characters long
next_char = [] # One element for each sequence

# For Loop
for i in range(0, len(encoded) - max_len, step):
  sequences.append(encoded[i : i + max_len])
  next_char.append(encoded[i + max_len])

# Creating X and y
X = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)


# For Loop:
# What do the dimensions of X represent?
# Number of Sequences; Sequence Length; Unique Characters
for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    X[i,t,char] = 1

  # What do the dimensions of y represent?
  # Number of Sequences; Unique Characters
  y[i, next_char[i]] = 1

In [ ]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

## Build LSTM Model
# Instantiating Model
lstm = Sequential([
  LSTM(128, input_shape=(max_len, len(chars))),
  Dense(len(chars), activation="softmax")
])

# Compiler - 'nadam' to retain momentum term
lstm.compile(loss="categorical_crossentropy", optimizer="nadam")

# Summary
lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               118272    
_________________________________________________________________
dense (Dense)                (None, 102)               13158     
Total params: 131,430
Trainable params: 131,430
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import random
import sys

# Sampling and per-epoch display functions from lecture
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - max_len - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + max_len]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, max_len, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
# Epoch-Printing Callback
from tensorflow.keras.callbacks import LambdaCallback
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Fitting
lstm.fit(X, y,
         batch_size=32,
         epochs=10,
         callbacks=[print_callback])

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN